# Text Summarization [BART Fine Tune]

## 01. Install and Import Libraries

In [ ]:
!pip install -U datasets transformers[torch] rouge-score evaluate accelerate nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=aeda08d16bdfe11347923e2b7a86cf017244ab04d1dac5e65e4a6f8922c7b670
  Store

In [ ]:
from huggingface_hub import notebook_login

# Paste hugging face token with write permission enabled and log in
notebook_login()

In [ ]:
import datasets
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("billsum")
metric = load("rouge")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/bart-large-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
max_input_length = 1024
max_target_length = 128

prefix = "summarize: "

# Preprocess input data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Dynamic Padding batch wise
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=model_checkpoint,
                                       label_pad_token_id=tokenizer.pad_token_id)

# Load Pre-Trained Bart-Large Checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]

# Define hyperparams
args = Seq2SeqTrainingArguments(
    f"bart-large-summarizer",
    num_train_epochs=3,                       # No of epochs to train
    per_device_train_batch_size=batch_size,   # Batch size for training per device
    per_device_eval_batch_size=batch_size,    # Batch size for evaluation per device
    learning_rate=2e-5,                       # Learning rate for optimizer
    warmup_steps=500,                         # No of warmup steps for the learning rate scheduler
    weight_decay=0.01,                        # Weight decay coefficient for regularization
    logging_dir='./logs',                     # Directory for logging training information
    load_best_model_at_end=True,              # Whether to load the best model from checkpoints at the end of training
    logging_steps=1000,                       # Log training metrics every `logging_steps` steps
    save_steps=1000,                          # Save model checkpoints every `save_steps` steps
    save_total_limit=2,                       # Save no of checkpoints
    eval_strategy="steps",                    # When to run evaluation during training: steps, epochs or none
    predict_with_generate=True,               # Generate preds during eval for metrics score
    generation_max_length=32,                 # Generation length | NOTE: Evaluation becomes faster if choosen small gen_length
    fp16=True,                                # Floating point 16 precision
    push_to_hub=True,                         # Save checkpoint in Hugging Face Hub
)

In [ ]:
import nltk
import numpy as np
import evaluate

rouge = evaluate.load("rouge")
nltk.download('punkt')
nltk.download('punkt_tab')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # RougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Replace -100 in labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"].select(range(2000)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
from accelerate import Accelerator

# Initialize Accelerator and Trainer
Accelerator()
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1000,1.705000,1.354412,33.885700,25.928400,31.620600,32.308200,31.998000
2000,1.322000,1.244997,34.177200,26.530200,32.031000,32.672500,32.000000
3000,1.158600,1.210188,34.487200,26.991200,32.402500,32.989600,31.983500
4000,1.099500,1.180635,34.732900,27.248900,32.587500,33.226600,31.997500
5000,1.047200,1.184374,34.775100,27.425500,32.663800,33.294100,31.989500
6000,0.966800,1.165147,34.918000,27.424000,32.732200,33.373200,31.991500


In [ ]:
trainer.push_to_hub()